In [7]:
import pandas as pd
import os
import yfinance as yf
import numpy as np
from scipy.stats import norm
import datetime

In [8]:
ticker = 'NVDA'
stock = yf.Ticker(ticker)
expiration_date = '2027-01-15' 

# Get current price
S = stock.history(period="1d")['Close'].iloc[-1]

# Get the options chain for the specified expiration date
options = stock.option_chain(expiration_date)
calls = options.calls
puts = options.puts

calls["OptionType"] = "Call"
puts["OptionType"] = "Put"

calls = calls[["strike", "lastPrice", "inTheMoney"]]
puts = puts[["strike", "lastPrice", "inTheMoney"]]

puts.rename(columns={"lastPrice": "PutPrice"}, inplace=True)
calls.rename(columns={"lastPrice": "CallPrice"}, inplace=True)

options = pd.merge(calls, puts, on="strike", how="outer")

expiration = datetime.datetime.strptime(expiration_date, '%Y-%m-%d')
today = datetime.datetime.today()
T = (expiration - today).days / 365.25

r = 0.0422

# Estimate volatility using historical data (annualized)
hist = yf.download(ticker, period="1y")
hist['returns'] = np.log(hist['Close'] / hist['Close'].shift(1))
sigma = np.std(hist['returns'].dropna()) * np.sqrt(252)

options["Stock_Price"] = S
options["Time_to_Expire"] = T
options["Risk_Free_Rate"] = r
options["IV"] = sigma

# Save to CSV for analysis
csv_file = "nvda_options_data.csv"
if not os.path.isfile(csv_file):
    options.to_csv(csv_file, index=False)
else:
    options.to_csv(csv_file, mode='a', header=False, index=False)



/var/folders/8j/kzmsmn5s4b3d1xt9pyn9zkp80000gn/T/ipykernel_33736/1084391196.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puts.rename(columns={"lastPrice": "PutPrice"}, inplace=True)
/var/folders/8j/kzmsmn5s4b3d1xt9pyn9zkp80000gn/T/ipykernel_33736/1084391196.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.rename(columns={"lastPrice": "CallPrice"}, inplace=True)
[*********************100%***********************]  1 of 1 completed
